# Übungsaufgaben Transaktionen

**Hinweis:** Die folgenden Aufgaben beziehen sich auf die WWI-Datenbank. 

Diverse Operationen sollen nun mit Transaktionen so abgesichtert werden, dass die Datenintegrität gewährleistet ist, also kein ungültiger Zustand der Daten entstehen kann.

**1\. Produkte bestellen**

1.1 Lagerbestand auffüllen

Der Geschäftskunde "Nils Kaulins" bestellt für seinen kleinen PC-Shop einige Fun-USB-Sticks: 

- 20 Stück "USB food flash drive - hot dog" 
- 10 Stück "USB food flash drive - banana". 

Erstelle dazu eine Transaktion mit allen nötigen Queries (Gerüst unten ergänzen):

**Neue Bestellung** in der Tabelle _Sales.Orders_

- Die _CustomerID_ von "Nils Kaulins" findest du in der Tabelle _Sales.Customers_. 
- Ebenso die _ContactPersonID_ (=_PrimaryContactPersonID_).
- Als _SalespersonPersonID_ und _LastEditedBy_ kannst du eine beliebige Person aus der Tabelle _Application.People_ wählen mit _IsSalesperson=1_.
- Als _CustomerPurchaseOrderNumber_ kannst du eine 5-stellige Zufallszahl generieren oder eine Fantasiezahl reinschreiben.
- _OrderDate_: Füge mit einer Funktion das aktuelle Datum ein.
- _ExpectedDeliveryDate_: Aktuelles Datum plus 3 Tage
- _IsUndersupplyBackordered_ = 1
- Die übrigen Felder kannst du weglassen bzw. NULL einfügen oder Werte von einer anderen bestehenden Bestellung kopieren.

**Zwei Bestell-Zeilen** in der Tabelle _Sales.OrderLines_.

- _OrderID_: ID der letzten Bestellung für den aktuellen Kunden (Tabelle _Sales.Orders_).   
    Tipp: Statt _INSERT INTO Sales.OrderLines (OrderID, ...) VALUES (1234, ....);_ kann man für 1234 eine Unterabfrage einsetzen und die ID dynamisch auslesen:   
    _INSERT INTO Sales.OrderLines (OrderID, ...) VALUES ((SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC), ....);_
- _StockItemID_ und übrige Felder: Suche eine bestehende Bestellung in _Sales.OrderLines_ mit _Description IN('USB food flash drive - hot dog', 'USB food flash drive - banana')_ und kopiere die Werte von da, ausser Quantity natürlich.
- _PickedQuantity_ soll erst mal 0 (null) sein, wird erst bei der Auslieferung gesetzt.
- _LastEditedBy_ ist gleich die bei der Bestellung oben

Teste dann einmal mit _ROLLBACK TRANSACTION_ und mit _COMMIT TRANSACTION_ und kontrolliere die neuen Tabellen-Einträge.

In [1]:
-- Transaktion starten
BEGIN TRANSACTION;

-- Bestellung
INSERT INTO Sales.Orders (
    CustomerID, SalespersonPersonID, ContactPersonID,
    OrderDate, ExpectedDeliveryDate, CustomerPurchaseOrderNumber, 
    IsUndersupplyBackordered, LastEditedBy
)
VALUES (
    1018, 2, 3218, GETDATE(), GETDATE() + 3, FLOOR(RAND() * 99999), 1, 2
);

-- Bestellzeilen
INSERT INTO Sales.OrderLines (
    OrderID, StockItemID, Description, PackageTypeID,
    Quantity, UnitPrice, TaxRate, PickedQuantity,
    LastEditedBy
) 
VALUES (
    (SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC), -- ID der letzten Bestellung des Kunden
    6, 'USB food flash drive - hot dog', 7, 20, 32.00, 15.00, 0, 2
), 
(
    (SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC), 
    9, 'USB food flash drive - banana', 7, 10, 32.00, 15.00, 0, 2
);

-- Transaktion abschliessen (ROLLBACK oder COMMIT)
COMMIT TRANSACTION;
-- Oder
-- ROLLBACK TRANSACTION;


(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.018

Kontrolliere vor und nachher die neuen Tabellen-Einträge:

In [2]:
-- Tabellen kontrollieren: Bestellungen
SELECT TOP (5) * FROM Sales.Orders  ORDER BY OrderID DESC;
SELECT TOP (10) * FROM Sales.OrderLines ORDER BY OrderLineID DESC;

(5 rows affected)

(10 rows affected)

Total execution time: 00:00:00.022

OrderID,CustomerID,SalespersonPersonID,PickedByPersonID,ContactPersonID,BackorderOrderID,OrderDate,ExpectedDeliveryDate,CustomerPurchaseOrderNumber,IsUndersupplyBackordered,Comments,DeliveryInstructions,InternalComments,PickingCompletedWhen,LastEditedBy,LastEditedWhen
73615,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,36684,1,NULL,NULL,NULL,NULL,2,2022-01-04 17:14:15.3910438
73614,1018,2,3218,3218,NULL,2022-01-04,2022-01-07,60915,1,NULL,NULL,NULL,2022-01-04 16:31:18.5466667,2,2022-01-04 16:31:18.5466667
73612,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,16892,1,NULL,NULL,NULL,NULL,2,2022-01-04 14:42:00.8292347
73611,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,77775,1,NULL,NULL,NULL,NULL,2,2022-01-04 14:35:14.1437853
73610,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,79374,1,NULL,NULL,NULL,NULL,2,2022-01-04 14:34:45.1860899


OrderLineID,OrderID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,PickedQuantity,PickingCompletedWhen,LastEditedBy,LastEditedWhen
231430,73615,9,USB food flash drive - banana,7,10,32.00,15.000,0,NULL,2,2022-01-04 17:14:15.3950132
231429,73615,6,USB food flash drive - hot dog,7,20,32.00,15.000,0,NULL,2,2022-01-04 17:14:15.3950132
231428,73614,9,USB food flash drive - banana,7,10,32.00,15.000,10,2022-01-04 17:10:46.0700000,2,2022-01-04 17:10:46.0700000
231427,73614,6,USB food flash drive - hot dog,7,20,32.00,15.000,20,2022-01-04 17:10:46.0700000,2,2022-01-04 17:10:46.0700000
231424,73612,9,USB food flash drive - banana,7,10,32.00,15.000,0,NULL,2,2022-01-04 14:42:00.8292347
231423,73612,6,USB food flash drive - hot dog,7,20,32.00,15.000,0,NULL,2,2022-01-04 14:42:00.8292347
231415,73602,9,USB food flash drive - banana,7,10,32.00,15.000,0,NULL,2,2022-01-04 14:29:33.2542765
231414,73602,6,USB food flash drive - hot dog,7,20,32.00,15.000,0,NULL,2,2022-01-04 14:29:33.2542765
231412,73586,130,Furry gorilla with big eyes slippers (Black) S,7,8,32.00,15.000,8,2016-05-31 11:00:00.0000000,4,2016-05-31 11:00:00.0000000
231411,73586,170,20 mm Anti static bubble wrap (Blue) 50m,7,40,102.00,15.000,40,2016-05-31 11:00:00.0000000,4,2016-05-31 11:00:00.0000000


**2\. Bestellungen ausliefern**

Erstelle alle Queries, um deine oben erstelle Bestellung nun auszuliefern: 

- Aktualisiere alle Bestellzeilen mit der _OrderID_ der letzten Bestellung des Kunden "Nils Kaulins" (gleich wie oben).
- Setze in der Tabelle _Sales.OrderLines_ das Feld _PickedQuantity_ auf den Wert von _Quantity_ und _PickingCompletedWhen_ und _LastEditedWhen_ auf das aktuelle Datum.

In [3]:
-- Bestellzeilen aktualisieren
UPDATE Sales.OrderLines SET 
    PickedQuantity = Quantity, 
    PickingCompletedWhen = GETDATE(), 
    LastEditedWhen = GETDATE()
WHERE OrderID = (
    SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
);

(2 rows affected)

Total execution time: 00:00:00.012

Die Kontaktperson hat die Ware selber abgeholt: In den Tabellen _Sales.Orders_ soll 

- das Feld _PickedByPersonID_ auf die _ContactPersonID_ und 
- das Feld _PickingCompletedWhen_ und _LastEditedWhen_ auf das aktuelle Datum gesetzt werden.

In [4]:
-- Bestellung aktualisieren
UPDATE Sales.Orders 
SET PickedByPersonID = ContactPersonID, 
    PickingCompletedWhen = GETDATE(), 
    LastEditedWhen = GETDATE() 
WHERE OrderID = (
    SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
)

(1 row affected)

Total execution time: 00:00:00.012

**3\. Reduziere dann den Lagerbestand** **(Zusatzaufgabe,** **freiwillig)**

- In _Warehouse.StockItemHoldings_ muss die Anzahl (_QuantityOnHand_) je Bestellzeile entsprechend reduziert werden. Wie du Werte aus einer Tabelle in eine andere kopierst, wird z.B. auf [dieser Webseite](https://www.monsterli.ch/blog/datenbank/sql-update-datensatz-erganzen-mit-einem-wert-aus-einer-anderen-tabelle/) erklärt. 
- Aktualisiere auch die Felder _LastEditedBy_ und _LastEditedWhen_ gleich wie bei den obigen Aufgaben.

**Tipp:** Wenn es zu schwierig ist, kannst du ausnahmsweise die benötigten IDs manuell aus den Tabellen auslesen und fix in deine Query rein schreiben.

In [5]:
-- Lagerbestand anpassen
-- Mit SELECT (Unterabfrage)
UPDATE Warehouse.StockItemHoldings 
SET QuantityOnHand = QuantityOnHand - ( 
        SELECT TOP (1) PickedQuantity FROM Sales.OrderLines 
        WHERE Warehouse.StockItemHoldings.StockItemID = Sales.OrderLines.StockItemID 
        AND Sales.OrderLines.OrderID = (
            SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
        ) 
    ), 
    LastEditedBy = 2, 
    LastEditedWhen = GETDATE()
-- Fix auf die 2 Produkte einschränken:
WHERE Warehouse.StockItemHoldings.StockItemID IN (6, 9);

-- Oder dynamisch aus der Bestellung auslesen:
WHERE Warehouse.StockItemHoldings.StockItemID IN (
    SELECT StockItemID FROM Sales.OrderLines 
    WHERE OrderID = (
        SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
    )
);

-- Mit JOIN
UPDATE Warehouse.StockItemHoldings 
SET QuantityOnHand = QuantityOnHand - Sales.OrderLines.PickedQuantity, 
    LastEditedBy = 2, 
    LastEditedWhen = GETDATE()
FROM Warehouse.StockItemHoldings 
    INNER JOIN Sales.OrderLines 
    ON Warehouse.StockItemHoldings.StockItemID = Sales.OrderLines.StockItemID
    AND Sales.OrderLines.OrderID = (
        SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
    );

: Msg 156, Level 15, State 1, Line 17
Falsche Syntax in der Nähe des WHERE-Schlüsselworts.

Total execution time: 00:00:00.002

Kontrolliere vor und nach deiner Query den Wert im Lagerbestand! Zeige dazu die Datensätze aus _Warehouse.StockItemHoldings_ mit den IDs 6 und 9 an.

In [6]:
-- Tabellen kontrollieren: Lagerbestand
SELECT StockItemID, QuantityOnHand, LastEditedWhen  
FROM Warehouse.StockItemHoldings 
WHERE StockItemID IN (6, 9);

(2 rows affected)

Total execution time: 00:00:00.006

StockItemID,QuantityOnHand,LastEditedWhen
6,196895,2022-01-04 17:10:46.0700000
9,192699,2022-01-04 17:10:46.0700000


**4\. Bestellungen sollen nur ausgeliefert werden können, wenn genügend Artikel an Lager sind**

Stelle nun sicher, dass obige Queries nur ausgefürt werden können, wenn noch genügend Artikel an Lager sind. Wenn etwas fehlt, soll alles rückgängig gemacht werden (ACID-Prinzip "Alles oder nichts"): 

- Erstelle auf der Tabelle _Warehouse.StockItemHoldings_ eine Restriction, welche sicherstellt, dass QuantityOnHand nie kleiner als 0 (null) wird.

In [7]:
ALTER TABLE Warehouse.StockItemHoldings 
ADD CONSTRAINT QuantityOnHand_check CHECK (QuantityOnHand >= 0);

Commands completed successfully.

Total execution time: 00:00:00.015

Kopiere nun die obigen Queries zusammen und verpacke sie in eine Transaktion, welche sicherstellt, dass alles nur ausgeführt wird, wenn kein Fehler passiert (z.B. Constraint nicht erfüllt). Wenn ein Fehler passiert, soll alles Rückgängig gemacht werden.  

- Verpacke die obigen UPDATE-Queries in einen TRY- / CATCH-Block.
- Ergänze die nötigen Transaktions-Befehle (BEGIN, COMMIT, ROLLBACK). Schau dazu das Beispiel in den Kurs-Unterlagen an.

In [8]:
-- Transaktion für Auslieferung starten
BEGIN TRANSACTION;

-- Versuch starten
BEGIN TRY

    -- Bestellung aktualisieren
    UPDATE Sales.OrderLines SET 
        PickedQuantity = Quantity, 
        PickingCompletedWhen = GETDATE(), 
        LastEditedWhen = GETDATE()
    WHERE OrderID = (
        SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
    );

    -- Bestellzeilen aktualisieren
    UPDATE Sales.OrderLines SET 
        PickedQuantity = Quantity, 
        PickingCompletedWhen = GETDATE(), 
        LastEditedWhen = GETDATE()
    WHERE OrderID = (
        SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
    );

    -- Lagerbestand anpassen
    UPDATE Warehouse.StockItemHoldings 
    SET QuantityOnHand = QuantityOnHand - ( 
            SELECT TOP (1) PickedQuantity FROM Sales.OrderLines 
            WHERE Warehouse.StockItemHoldings.StockItemID = Sales.OrderLines.StockItemID 
            AND Sales.OrderLines.OrderID = (
                SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
            ) 
        ), 
        LastEditedBy = 2, 
        LastEditedWhen = GETDATE()
    WHERE Warehouse.StockItemHoldings.StockItemID IN (
        SELECT StockItemID FROM Sales.OrderLines 
        WHERE OrderID = (
            SELECT TOP (1) OrderID FROM Sales.Orders WHERE CustomerID = 1018 ORDER BY OrderID DESC
        )
    );

    -- Transaktion abschliessen
    COMMIT TRANSACTION;

    -- Erfolgsmeldung ausgeben
    SELECT 'COMMITED: Produkte wurden erfolgreich ausgeliefert!' AS Meldung
END TRY

-- Fehlerbehandlung
BEGIN CATCH
    -- Transaktion rückgängig machen
    ROLLBACK TRANSACTION;

    -- Fehlermeldung ausgeben
    SELECT 'ROLLBACK: Zu wenige Produkte an Lager!' AS Meldung
END CATCH

GO

-- Tabellen kontrollieren: Bestellungen
SELECT TOP (5) * FROM Sales.Orders  ORDER BY OrderID DESC;
SELECT TOP (10) * FROM Sales.OrderLines ORDER BY OrderLineID DESC;

-- Tabellen kontrollieren: Lagerbestand
SELECT StockItemID, QuantityOnHand, LastEditedWhen  
FROM Warehouse.StockItemHoldings 
WHERE StockItemID IN (6, 9);

GO


(2 rows affected)

(2 rows affected)

(2 rows affected)

(1 row affected)

(5 rows affected)

...

Total execution time: 00:00:00.063

Meldung
COMMITED: Produkte wurden erfolgreich ausgeliefert!


OrderID,CustomerID,SalespersonPersonID,PickedByPersonID,ContactPersonID,BackorderOrderID,OrderDate,ExpectedDeliveryDate,CustomerPurchaseOrderNumber,IsUndersupplyBackordered,Comments,DeliveryInstructions,InternalComments,PickingCompletedWhen,LastEditedBy,LastEditedWhen
73615,1018,2,3218,3218,NULL,2022-01-04,2022-01-07,36684,1,NULL,NULL,NULL,2022-01-04 17:14:26.3433333,2,2022-01-04 17:14:26.3433333
73614,1018,2,3218,3218,NULL,2022-01-04,2022-01-07,60915,1,NULL,NULL,NULL,2022-01-04 16:31:18.5466667,2,2022-01-04 16:31:18.5466667
73612,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,16892,1,NULL,NULL,NULL,NULL,2,2022-01-04 14:42:00.8292347
73611,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,77775,1,NULL,NULL,NULL,NULL,2,2022-01-04 14:35:14.1437853
73610,1018,2,NULL,3218,NULL,2022-01-04,2022-01-07,79374,1,NULL,NULL,NULL,NULL,2,2022-01-04 14:34:45.1860899


OrderLineID,OrderID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,PickedQuantity,PickingCompletedWhen,LastEditedBy,LastEditedWhen
231430,73615,9,USB food flash drive - banana,7,10,32.00,15.000,10,2022-01-04 17:14:44.0833333,2,2022-01-04 17:14:44.0833333
231429,73615,6,USB food flash drive - hot dog,7,20,32.00,15.000,20,2022-01-04 17:14:44.0833333,2,2022-01-04 17:14:44.0833333
231428,73614,9,USB food flash drive - banana,7,10,32.00,15.000,10,2022-01-04 17:10:46.0700000,2,2022-01-04 17:10:46.0700000
231427,73614,6,USB food flash drive - hot dog,7,20,32.00,15.000,20,2022-01-04 17:10:46.0700000,2,2022-01-04 17:10:46.0700000
231424,73612,9,USB food flash drive - banana,7,10,32.00,15.000,0,NULL,2,2022-01-04 14:42:00.8292347
231423,73612,6,USB food flash drive - hot dog,7,20,32.00,15.000,0,NULL,2,2022-01-04 14:42:00.8292347
231415,73602,9,USB food flash drive - banana,7,10,32.00,15.000,0,NULL,2,2022-01-04 14:29:33.2542765
231414,73602,6,USB food flash drive - hot dog,7,20,32.00,15.000,0,NULL,2,2022-01-04 14:29:33.2542765
231412,73586,130,Furry gorilla with big eyes slippers (Black) S,7,8,32.00,15.000,8,2016-05-31 11:00:00.0000000,4,2016-05-31 11:00:00.0000000
231411,73586,170,20 mm Anti static bubble wrap (Blue) 50m,7,40,102.00,15.000,40,2016-05-31 11:00:00.0000000,4,2016-05-31 11:00:00.0000000


StockItemID,QuantityOnHand,LastEditedWhen
6,196875,2022-01-04 17:14:44.0833333
9,192689,2022-01-04 17:14:44.0833333
